In [460]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import unicodedata as ud
import re



In [461]:
top_price_table = pd.DataFrame.from_csv('price_table.csv')
doe_price_table = pd.DataFrame.from_csv('doe_values.csv')
com_price_table = pd.DataFrame.from_csv('com_values.csv')
for_hire_table = pd.DataFrame.from_csv('for_hire.csv')
private_fleet_table = pd.DataFrame.from_csv('private_fleet.csv')
logistics_table = pd.DataFrame.from_csv('logistics.csv')
freight_table = pd.DataFrame.from_csv('global_freight.csv')

In [462]:
def convert_to_hundreds(value):
    if value > 10.0:
        return value / 100.0
    else:
        return value

for key in doe_price_table.columns[3:13]:
    doe_price_table[key] = doe_price_table[key].\
        apply(convert_to_hundreds)
for key in com_price_table.columns[3:6]:
    com_price_table[key] = com_price_table[key].\
        apply(convert_to_hundreds)


In [463]:
com_price_table = com_price_table[com_price_table.Year != 2000]
doe_price_table = doe_price_table[doe_price_table.Year != 2000]

In [464]:
index_number = np.arange(1,com_price_table.shape[0]+1)
com_price_table.index = index_number
index_number = np.arange(1,doe_price_table.shape[0]+1)
doe_price_table.index = index_number
if len(com_price_table['Week Ending'] == '12/30/2000-1/5/2001') != 0:
    index1 = com_price_table[com_price_table['Week Ending'] == '12/30/2000-1/5/2001'].index
    index2 = com_price_table.iloc[[-1]].index
    com_price_table.loc[index1[0]:index2[0],'Year'] = '2000'
#    print "1: %s, 2: %s" % (com_price_table.loc[index1[0]:index2[0],'Year'],  'junk')
if len(doe_price_table['Date'] == '1/1/2001') != 0:
    index1 = doe_price_table[doe_price_table['Date'] == '1/1/2001'].index
    index2 = doe_price_table.iloc[[-1]].index
    doe_price_table.loc[index1[1]:index2[0],'Year'] = '2000'
#    print "1: %s, 2: %s" % (doe_price_table.loc[index1[1]:index2[0],'Year'],  'junk')

In [465]:

def CleanupDates(data, key1, key2, set_col):    
    for index, item, year in zip(data.index,data[key1], data[key2]):
        count = len(re.findall(r"/", item))
        if count > 1:
#        print "Item is - %s %s %s" %(year, item, re.findall(r"/", item))
            if count == 2:
#            print "Item 2 - %s %s %s" %(year, item, re.findall(r"/", item))
                if len(re.findall(r"-", item)) == 1:
                #3/19 - 3/23, 12/26 - 12/30, 7/2 - 7/6
                    item = item[item.rfind("-")+1:]
#                print "Two week  format: %s %s" % (item, year)
                else:
#                print "Year format: %s %s" % (item, year)
                    if item[-3] == '/':
                    #1/4/13, 1/7/08, 12/26/11, 12/28/09, 1/2/06
#                    print item
                        item = item[: item.rfind("/")]
#                    print "Two digit Year format: %s %s" % (item, year)
                    else:
                    #1/1/2001, 12/29/2003, 9/1/2003
#                    print item
                        item = item[: item.rfind("/")]
#                    print "Four digit Year format: %s %s" % (item, year)                    
            elif count == 3:
            #1/3 - 1/7/11, 12/28 - 1/1/10, 12/29 - 1/2/09, 1/3-1/7/2005
#            print item
                item = item[item.rfind("-")+1: item.rfind("/")]
#            print "Item 3 - %s %s %s" %(year, item, re.findall(r"/", item))
            elif count == 4:
            #1/2/12 - 1/6/12, 1/5/09 - 1/9/09, 1/7/08 - 1/11/08, 12/31/07 - 1/4/08
            #12/29/2003-1/2/2004, 12/30/2002-1/04/2003, 12/31/2001-1/05/2002
            #12/30/2000-1/5/2001, 
#            print item
                item = item[item.rfind("-")+1: item.rfind("/")]
#            print "Item 4 - %s %s %s" %(year, item, re.findall(r"/", item))
            else:
                item = '1/1/1969'
#            print "Unkown"

        else:
#9/9-9-13, 4/22-28, 5/6-12, 5/13-19
            if len(re.findall(r"-", item)) == 1:
                item1 = item[:item.rfind("/")+1]
                item2 = item[item.rfind("-")+1:]
#            print "Special case %s %s" %(item1, item2)
                item = item1+item2
            if item == '9/9-9-13':
                item = '9/13'
        full_date = item +'/' + str(int(year))
        data.iloc[index-1, data.columns.get_loc(set_col)] = full_date
#        print " %s %s %s" % (index, full_date, data.iloc[index-1, data.columns.get_loc(set_col)] )


In [466]:

com_price_table['FullDate'] = 0
#print com_price_table['FullDate']
CleanupDates(com_price_table, 'Week Ending', 'Year', 'FullDate')    
#9/9-9-13/2002
doe_price_table['FullDate'] = 0
CleanupDates(doe_price_table, 'Date', 'Year', 'FullDate')    


In [467]:
com_price_table.to_csv('com_price_table_clean.csv')
doe_price_table.to_csv('doe_price_table_clean.csv')